In [1]:
""" Imports """
from natasha import (
    Segmenter,
    MorphVocab,
    Doc,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger
)
from yargy import rule, Parser, or_
from yargy.predicates import gram, dictionary
from yargy.interpretation import fact
import json
from IPython.display import display, Markdown

### Инициализация сущностей Natasha

In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

### Обработка текста

In [3]:
def text_processing(param_text):
    doc = Doc(param_text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    
    return doc

### Открытие файла с текстом

In [4]:
def read_file(path: str):
    with open(path, 'r', encoding='utf-8') as file:
        return file.read()

### Открытие файла со словарём

In [5]:
with open('geo_terms.json', encoding='utf-8') as f:
    geo_terms = json.load(f)

### Географические сущности

In [6]:
GeoMention = fact('GeoMention', ['location'])
ADJ = gram('ADJF')
NOUN = gram('NOUN')
GEO_TYPES_COUNTRY = dictionary(geo_terms['country'])
GEO_TYPES_REGION = dictionary(geo_terms["region"] + geo_terms["region_type"])
GEO_TYPES_OBJECT = dictionary(geo_terms["region"] + geo_terms["region_type"] + geo_terms["object"])

geo_rule = rule(
    ADJ.optional().repeatable().interpretation(GeoMention.location),
    or_(GEO_TYPES_COUNTRY, GEO_TYPES_REGION, GEO_TYPES_OBJECT)
).interpretation(GeoMention)
geo_parser = Parser(geo_rule)

### Инициализация парсера

In [7]:
def parsing(param_doc):
    for sent in param_doc.sents:
        sentence_text = sent.text

        # Геосущности из Natasha (LOC)
        natasha_locations = []
        for span in sent.spans:
            if span.type == 'LOC':
                natasha_locations.append(span.text)

        # 1. Гео-сущности через Yargy
        geo_mentions = []
        for match in geo_parser.findall(sentence_text):
            geo_text = sentence_text[match.span.start:match.span.stop]
            if any(term.lower() in geo_text.lower() for term in
                   geo_terms["country"]
                   or geo_terms["region_type"]
                   or geo_terms["region"] + geo_terms["region_type"]
                   or geo_terms["region"] + geo_terms["region_type"] + geo_terms["object"]):
                geo_mentions.append(geo_text.strip())

        # Natasha NER
        for span in sent.spans:
            if span.type == 'LOC':
                geo_mentions.append(span.text.strip())

        # 2. Поиск действия (глагол + [опционально ADJF/PRTF] + NOUN, справа или слева)
        actions = []
        for i, token in enumerate(sent.tokens):
            if 'VERB' in token.pos or 'PRTS' in token.pos:
                verb = token.text

                # === ПОИСК СУЩЕСТВИТЕЛЬНОГО ===

                # Сначала ищем NOUN справа
                noun_idx = next((j for j, t in enumerate(sent.tokens[i + 1:], start=i + 1) if 'NOUN' in t.pos), None)

                # Если не нашли — ищем NOUN слева
                if noun_idx is None:
                    noun_idx = next((j for j, t in enumerate(reversed(sent.tokens[:i])) if 'NOUN' in t.pos), None)
                    if noun_idx is not None:
                        noun_idx = i - noun_idx - 1  # пересчёт индекса в нормальный

                # Если нашли существительное
                if noun_idx is not None:
                    noun_token = sent.tokens[noun_idx]
                    phrase_tokens = [noun_token.text]

                    # === ПРИСОЕДИНЯЕМ ПРИЛАГАТЕЛЬНЫЕ/ПРИЧАСТИЯ ===
                    # Сперва смотрим слева от существительного (в том же порядке, как в предложении)
                    for j in range(noun_idx - 1, -1, -1):
                        t = sent.tokens[j]
                        if 'ADJF' in t.pos or 'PRTF' in t.pos or 'PRTS' in t.pos:
                            phrase_tokens.insert(0, t.text)
                        else:
                            break  # как только не согласующееся — выходим

                    action_phrase = f"{verb} {' '.join(phrase_tokens)}"
                    actions.append(action_phrase)

        # 3. Привязка
        unique_pairs = set()
        for geo in geo_mentions:
            action_text = actions[0] if actions else '(действие не найдено)'
            pair = f"{geo} — {action_text}"
            if pair not in unique_pairs:
                unique_pairs.add(pair)
                display(Markdown(pair))

### Прогон парсера

In [8]:
def start_parsing():
    
    for i in range(1, 6):
        text = read_file(f"resources/example_text_{i}.txt")
        display(Markdown(f""" #### Текст №{i} """))
        parsing(text_processing(text))

# Результаты

In [9]:
start_parsing()

 #### Текст №1 

Ставропольском крае — открыт терминал

Минеральные Воды — открыт терминал

Курганской области — открыт завод

Лебедянском районе — открыт завод

...

Пятигорске — реализуется проект

Западном — реализуется проект

Кисловодске — ведётся строительство

Самарской области — выявленных территориях

Чапаевске — выявленных территориях